In [2]:

import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.utils import resample
from statsmodels.tsa.stattools import grangercausalitytests

# Define true causal relationships
true_coefficients = {
    'business_confidence': 0.3,  # Business confidence influences real GDP
    'consumer_confidence': 0.2,  # Consumer confidence influences real GDP
    'cboe_volatility': -0.1,     # CBOE volatility influences real GDP negatively
}

# Generate synthetic data based on true causal relationships
def generate_synthetic_data(n_points=100, lags=3, true_coeffs=true_coefficients):
    np.random.seed(42)
    df_synthetic = pd.DataFrame(np.random.normal(0, 1, (n_points, len(true_coeffs) + 1)),
                                columns=['real_gdp'] + list(true_coeffs.keys()))

    for i in range(lags, n_points):
        for var, coeff in true_coeffs.items():
            df_synthetic.at[i, 'real_gdp'] += coeff * df_synthetic.at[i-lags, var]

        # Add some noise
        df_synthetic.at[i, 'real_gdp'] += np.random.normal(0, 0.1)

    return df_synthetic

# Generate synthetic dataset
synthetic_data = generate_synthetic_data()

In [3]:
# Function to perform Granger causality tests
def perform_granger_causality_tests(data, target='real_gdp', maxlag=12):
    causality_results = {}
    for col in data.columns:
        if col != target:
            test_result = grangercausalitytests(data[[target, col]], maxlag=maxlag, verbose=False)
            causality_results[col] = [round(test_result[i+1][0]['ssr_ftest'][1], 4) for i in range(maxlag)]
    return causality_results


In [4]:
# Perform bootstrap Granger causality tests
def bootstrap_granger_causality(data, target='real_gdp', n_bootstrap=100, maxlag=12):
    bootstrap_results = {col: [] for col in data.columns if col != target}
    for _ in range(n_bootstrap):
        boot_data = resample(data)
        gc_results = perform_granger_causality_tests(boot_data)
        for col, p_values in gc_results.items():
            bootstrap_results[col].append(p_values)
    
    confidence_intervals = {}
    for col, values in bootstrap_results.items():
        values = np.array(values)
        ci_lower = np.percentile(values, 2.5, axis=0)
        ci_upper = np.percentile(values, 97.5, axis=0)
        confidence_intervals[col] = (ci_lower, ci_upper)
    
    return confidence_intervals

# Generate bootstrap confidence intervals
bootstrap_cis = bootstrap_granger_causality(synthetic_data)

print("Bootstrap Confidence Intervals (Synthetic Data):")
for key, (ci_lower, ci_upper) in bootstrap_cis.items():
    print(f"{key} causing real_gdp: CI Lower={ci_lower}, CI Upper={ci_upper}")

c:\Users\ellac\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\ellac\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\ellac\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\ellac\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\ellac\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  

Bootstrap Confidence Intervals (Synthetic Data):
business_confidence causing real_gdp: CI Lower=[0.03067   0.0404975 0.04462   0.021045  0.0336725 0.022405  0.0261025
 0.0294725 0.031365  0.0470725 0.0334875 0.04117  ], CI Upper=[0.97538   0.9703925 0.9720325 0.99042   0.9606625 0.962635  0.9675275
 0.981855  0.9788775 0.9906675 0.9929625 0.9963825]
consumer_confidence causing real_gdp: CI Lower=[0.0781    0.02623   0.040345  0.03682   0.04367   0.0344275 0.0354275
 0.025065  0.0146925 0.020665  0.0178475 0.0182725], CI Upper=[0.9820775 0.9723075 0.9834225 0.95315   0.9824525 0.991415  0.995615
 0.98625   0.98731   0.9856625 0.9627275 0.976085 ]
cboe_volatility causing real_gdp: CI Lower=[0.015065  0.0141475 0.026865  0.021985  0.035395  0.041135  0.0370375
 0.02863   0.036975  0.02096   0.0227425 0.0177275], CI Upper=[0.976575  0.95753   0.94535   0.9546175 0.950885  0.93567   0.965275
 0.97351   0.9630875 0.95579   0.962705  0.9487325]


c:\Users\ellac\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\ellac\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
